<a href="https://colab.research.google.com/github/hmghaly/word_align/blob/master/walign_mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
from google.colab import drive
drive.mount('/content/drive/')
cwd="/content/drive/My Drive/Colab Notebooks"
os.chdir(cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#Downloading something from the server
!wget "http://arbsq.net/uncorpus/ar-en/UNv1.0.ar-en-clean2.ar"

--2020-04-16 16:56:31--  http://arbsq.net/uncorpus/ar-en/UNv1.0.ar-en-clean2.ar
Resolving arbsq.net (arbsq.net)... 184.168.246.126
Connecting to arbsq.net (arbsq.net)|184.168.246.126|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4458094300 (4.2G)
Saving to: ‘UNv1.0.ar-en-clean2.ar.1’

UNv1.0.ar-en-clean2 100%[===================>]   4.15G  20.1MB/s    in 3m 28s  

2020-04-16 16:59:59 (20.4 MB/s) - ‘UNv1.0.ar-en-clean2.ar.1’ saved [4458094300/4458094300]



In [ ]:
#uploading something to the server
import ftplib, os
local_dir=""
local_fname="UNv1.0.ar-en-clean-tok.ar"
remote_dir="uncorpus/ar-en"
local_fpath= os.path.join(local_dir,local_fname)
remote_fpath= os.path.join(remote_dir,local_fname)

session = ftplib.FTP('champolu.com','arbsquser','Abc1234!')
file = open(local_fpath,'rb')                  # file to send
session.storbinary('STOR %s'%remote_fpath, file)     # send the file
file.close()                                    # close file and FTP
session.quit()

def upload(local_dir,local_fname,remote_dir,server_name="",username="",password=""): #upload a file to the server
  local_fpath= os.path.join(local_dir,local_fname)
  remote_fpath= os.path.join(remote_dir,local_fname)

  #session = ftplib.FTP(server_name,username,password)
  session = ftplib.FTP('champolu.com','arbsquser','Abc1234!')
  file = open(local_fpath,'rb')                  # file to send
  session.storbinary('STOR %s'%remote_fpath, file)     # send the file
  file.close()                                    # close file and FTP
  session.quit()


'221-Goodbye. You uploaded 4974896 and downloaded 0 kbytes.\n221 Logout.'

In [ ]:
#main functions
#encoding large integers for line locations into fixed length strings for easy retreival
import sys, os, re
from time import time

excluded_src_tokens=["the","a","an","and","of","its","his","her","to","is","are","am","been", "have","has","had", "in", "on", "at", "with", "from","for","that","he","she","it"] #we will need to update this
excluded_trg_tokens=["من","على","في","مع", "إلى","له","تم","إلى","في"]


def encode_num(int_number,size=6,base=128):
  num=int_number
  encoded=""
  while num>0:
    div,rem=(int(num/base), num%base)
    our_char=chr(rem)#.encode("utf-8")
    num=div
    encoded+=our_char
  encoded+=chr(0)*(size-len(encoded))
  return encoded

def decode_num(encoded_str,base=128):
  total=0
  for i,c in enumerate(encoded_str):
    total+=ord(c)*base**i
  return total

def number_file_lines(main_file_path,numbering_file_path):
  if sys.version[0]=="2": file_locs_fopen=open(numbering_file_path,"w")
  else: file_locs_fopen=open(numbering_file_path,"w",newline="")
  
  main_fopen=open(main_file_path,"r")
  #i_=0
  while True:
    loc=main_fopen.tell()
    line=main_fopen.readline()
    encoded_loc=encode_num(loc)
    file_locs_fopen.write(encoded_loc)
    if line=="": break

  main_fopen.close()
  file_locs_fopen.close()

def get_line(line_num, main_file_obj,locs_file_obj,str_size=6):
  lookup_i=line_num*str_size
  locs_file_obj.seek(lookup_i)
  chars=locs_file_obj.read(str_size)
  file_loc=decode_num(chars)
  main_file_obj.seek(file_loc)
  file_line=main_file_obj.readline()
  return file_line

def tok(txt):
  #txt=txt.replace(u'\x01'," ")
  txt=txt.replace(u'\u2019',"'")
  
  txt=txt.replace("'s ","_s ")
  txt=txt.replace("'re ","_re ")
  txt=txt.replace("can't ","cann_t ")
  txt=txt.replace("n't ","n_t ")
  txt=re.sub("(?u)(\W)",r" \1 ", txt)
  txt=txt.replace("_s ", " 's ")
  txt=txt.replace("_re "," 're ")
  txt=txt.replace("n_t "," n't ")
  
  out=re.split("\s+",txt)
  return [v for v in out if v]

def rtrv_idx(tokens,idx_dict):
    cur_indexes=idx_dict.get(tokens[0],[])
    for ti,ta in enumerate(tokens[1:]):
        cur_offset=ti+1
        cur_raw_idx=idx_dict.get(ta,[])
        cur_offset_idx=[(v[0],v[1]-cur_offset) for v in cur_raw_idx]
        cur_indexes=list(set(cur_indexes).intersection(set(cur_offset_idx)))
    cur_indexes=[v[0] for v in cur_indexes]
    cur_indexes=sorted(list(set(cur_indexes)))
    return cur_indexes

def idx_match(idx1,idx2): #This function is to give a ratio of matching between the inverted indexes for two words
    if len(idx1)==0 or len(idx2)==0: return 0
    intersection=len(list(set(idx1).intersection(set(idx2))))
    union=len(idx1)+len(idx2)-intersection
    if union==0: return 0
    return float(intersection)/union

def match_sentences_indexes(cur_src,cur_trg,src_inverted_index,trg_inverted_index):
  #excluded_src_tokens=["the","a","an","and","of","its","his","her","to","is","are","am","with"] #we will need to update this
  #excluded_trg_tokens=[]

  #src_sent=["<s>"]+cur_src+["</s>"] #adding start and end sentences
  #trg_sent=["<s>"]+cur_trg+["</s>"]
  src_sent=cur_src
  trg_sent=cur_trg

  #array1=np.zeros((len(trg_sent), len(src_sent))) #starting to populate the numpy probability array with zeros
  wt_list=[]
  #TODO: optimize the code for using sentence indexes for words, to save calculations for words occuring more than once in the sentence
  for j_ in range(len(trg_sent)): #now we identify the probability of matching between each pair of src and trg words
    t_tok=trg_sent[j_]
    #TODO: do generic steps for exluding stopwords in both source and target words
    if "ـ" in t_tok: t_indexes=[] #if an Arabic word has tatweel in our tokenization scheme it is a prefix or suffix, so it is excluded from matching
    elif t_tok in excluded_trg_tokens: t_indexes=[] #if target token is a stop word in target tokens (excluded_trg_tokens)
    else: t_indexes=trg_inverted_index.get(t_tok,[])    #else get the indexes from the inverted index
    for i_ in range(len(src_sent)):
      s_tok=src_sent[i_]
      if s_tok==t_tok and s_tok in ["<s>","</s>"]: ratio=1
      elif s_tok in excluded_src_tokens: ratio=0
      else: 
        s_indexes=src_inverted_index.get(s_tok,[])
        ratio=idx_match(s_indexes,t_indexes)
      
      #array1[i_,j_]=ratio #we populate the array with the matching ratio - Maybe not needed, except for visualization, when available
      wt_list.append(((i_,j_),ratio)) #and also add the coordinates and the ratio to a list
  
  #print(array1)
  wt_list.sort(key=lambda x:-x[1]) #we sort the weight list by the probability of each coordinate
  return wt_list #return wt_list and matching probability list

def get_candidate_pts(wt_list,src_size,trg_size): #or some dictionary
  final_list=[]
  src_side_dict=dict(iter([(i,[0]*trg_size) for i in range(src_size)] )) #dict with keys corresponding to src sent word indexes, and values are the probability of each target word index
  trg_side_dict=dict(iter([(i,[0]*src_size) for i in range(trg_size)] )) #dict with keys corresponding to trg

  #trg_side_dict=dict(iter([(i,[0]*trg_size) for i in range(src_size)] )) #dict with keys corresponding to src sent word indexes, and values are the probability of each target word index
  #src_side_dict=dict(iter([(i,[0]*src_size) for i in range(trg_size)] )) #dict with keys corresponding to trg


  #we collect the coordinates with the highest probability across the matrix, with unique x,y/i,j coordinates
  used_i,used_j=[],[] #if an index (i or j) is used once in a coordinate, it is excluded 
  #final_list=[]
  for ij,wt in wt_list: #while we go in a descending order in all the coordinates with their weights
    if wt==0: continue
    i0,j0=ij
    src_side_dict[i0][j0]=wt
    trg_side_dict[j0][i0]=wt

    #if i0==0 and j0!=0: continue
    #if i0!=0 and j0==0: continue
    if i0 in used_i or j0 in used_j: continue
    #if i0 in used_i and j0 in used_j: continue
    if not (i0,j0) in final_list: 
      final_list.append((i0,j0)) #these unique coordinates are also added to the final candidate points
      #array1[i0,j0]+=25
    used_i.append(i0)
    used_j.append(j0)

    for i_ in range(src_size): #get the maximum coordinates corresponding to each source word
      corr=src_side_dict[i_]
      max_corr=max(corr)
      if max_corr==0: continue
      max_j_indexes=[k0 for k0,v in enumerate(corr) if v==max_corr]
      for j_ in max_j_indexes:
        final_list.append((i_,j_))

    for j_ in range(trg_size): #and the maximum corresponding to each target word
      corr=trg_side_dict[j_]
      max_corr=max(corr)
      if max_corr==0: continue
      max_i_indexes=[k0 for k0,v in enumerate(corr) if v==max_corr]
      for i_ in max_i_indexes:
        final_list.append((i_,j_)) 
  final_list=list(set(final_list)) #now we have the final list of possible points, that we can process with dynamic programming
  final_list.sort()
  return final_list #return candidate points, which are maximum points in rows and columns and overall

def get_path_rect(se_span_dict,ne_span_dict,pt_dict,se): #se: south east, if the rectangles are expanding towards South East or not (towards NE: north east)
  #se= False
  if se: cur_span_dict=se_span_dict
  else: cur_span_dict=ne_span_dict
  keys_wts=list(cur_span_dict.items())

  path_wt_dict={} #the weight of the path till a certail span
  prev_span_dict={} #the previous span with the highest path weight to a given span

  for cur_span,cur_wt in keys_wts:
    (cur_x0,cur_y0),(cur_x1,cur_y1)=cur_span
    cur_span_path_wt=path_wt_dict.get(cur_span,cur_wt) #we check the path weight so far for the current span - if nothing, then it is the actual weight
    path_wt_dict[cur_span]=cur_span_path_wt  #we then update it
    for next_span,next_wt in keys_wts:
      (next_x0,next_y0),(next_x1,next_y1)=next_span
      #accept only spans going either SE or NE - or colinear spans
      colinear=False #check if two spans are colinear on X or Y
      if cur_span==next_span: continue #eliminate if same span
      if next_x0<cur_x1: continue #eliminate if the start x of the next span is less than the end x of current
      if se and next_y0<cur_y1: continue
      if not se and next_y0>cur_y1: continue
      if next_x0==cur_x1:
        if cur_x0==cur_x1==next_x0==next_x1: 
          colinear=True #msg="colinear X"
          if abs(next_y0-cur_y1)>5: continue #maximum number of words between colinear spans
        else: continue
      if next_y0==cur_y1:
        if cur_y0==cur_y1==next_y0==next_y1: 
          colinear=True # msg="colinear Y"
          if abs(next_x0-cur_x1)>5: continue #maximum number of words between colinear spans
        else: continue    
      # col_penalty=0.8 #not sure if we'll use these
      # if colinear: next_wt=col_penalty*next_wt #apply penalty to colinear spans
      combined_span_wt=cur_span_path_wt+next_wt #include conditions for colinarity
      found_combined_span_path_wt=path_wt_dict.get(next_span,0)
      if combined_span_wt>found_combined_span_path_wt:
        path_wt_dict[next_span]=combined_span_wt
        prev_span_dict[next_span]=cur_span
  #now processing the paths from each point
  used=[] #groups of spans along a rectangle that were used before
  for k,v in prev_span_dict.items(): 
    found=v
    path=[k]
    if found: path.append(found)
    while found:
      found=prev_span_dict.get(found)
      if found: path.append(found)
    cur_path=list(reversed(path))
    cur_wts=[cur_span_dict[v] for v in cur_path]
    for i0 in range(len(cur_path)):
      for i1 in range(i0+1,len(cur_path)):
        slice_spans=cur_path[i0:i1+1]
        if slice_spans in used: continue
        used.append(slice_spans)
        span1,span2=slice_spans[0],slice_spans[-1]
        slice_pts=[]
        for sp in slice_spans: 
          found_pts=pt_dict[sp]
          slice_pts.extend(found_pts)

        mixed_spans=span1+span2 #calculate the window of the two spans
        mixed_spans_xs=[v[0] for v in mixed_spans]
        mixed_spans_ys=[v[1] for v in mixed_spans]
        min_x,max_x=min(mixed_spans_xs),max(mixed_spans_xs)
        min_y,max_y=min(mixed_spans_ys),max(mixed_spans_ys)

        full_se_span=((min_x,min_y),(max_x,max_y))
        full_ne_span=((min_x,max_y),(max_x,min_y))

        found_se_wt=se_span_dict.get(full_se_span,0)
        found_ne_wt=ne_span_dict.get(full_ne_span,0)

        slice_wts=cur_wts[i0:i1+1]
        slice_total_wt=sum(slice_wts)
        if slice_total_wt> found_se_wt and slice_total_wt>found_ne_wt: #if the newly created span has more weight that what is found in se_dict or ne_dict, update
          # print("creating a new span")
          # print("full_se_span",full_se_span, "full_ne_span", full_ne_span, "weight:", slice_total_wt)
          # print("Points:",slice_pts)
          se_span_dict[full_se_span]=slice_total_wt
          ne_span_dict[full_ne_span]=slice_total_wt
          pt_dict[full_se_span]=slice_pts
          pt_dict[full_ne_span]=slice_pts
  return se_span_dict, ne_span_dict, pt_dict



def phrase_extraction(src,trg,solution,phrase_size_limit=10):
  #This function accepts the source and target tokenized sentences
  #in addition to the alignment solution, showing the mapping between src and trg tokens 
  #and the maximum size of a source phrase (in tokens)
  #solution=[(j,i) for i,j in solution] #we will need to check this from the input, for some reason the dynamix programming algorithms switches the x,y (i,j) src/trg coordinates

  used_j=sorted(list(set([v[1] for v in solution]))) #we need this to extend the identified target phrase with surrounding unaligned points
  all_phrase_pairs=[]

  for i0 in range(len(src)): #iterating over the source sentence, similar to Koehn's algorithm
    for i1 in range(i0,len(src)):
      if i1-i0>phrase_size_limit-1: break
      #if i0==i1: continue
      accept=True
      j0,j1=len(trg),0 #initializing the target span 
      for ii,jj in solution:
        if ii>=i0 and ii<=i1: 
          if jj<j0: j0=jj #getting the minimum target point within the current source span (i0,i1)
          if jj>j1: j1=jj #and also the maximum point 

      if j1==0: accept=False #if there are no points within the source span, we don't accept the span 
      
      for ii,jj in solution: #and then we check if there are points within the identified target span (j0,j1) that fall outside the source span
        if jj>=j0 and jj<=j1: 
          if ii<i0 or ii>i1: 
            accept=False
            break        
      if accept: #if now the source span and target span are acceptable, we proceed
        #print("SRC:", i0,i1,src[i0:i1+1])
        src_phrase=" ".join(src[i0:i1+1]) #This is the source phrase, based on the sentence tokens and the source span
        #print("TRG:",j0,j1,trg[j0:j1+1], accept)

        #and then based on the list of used j indexes, we try to identify the previous aligned j-coordinate and the next
        #in order to extend the span for unalign points >>> I think there is an easier way
        used_j_before=[v for v in used_j if v<j0] 
        used_j_after=[v for v in used_j if v>j1]
        if used_j_before: #so we identify the starting points of the extended span
          prev_j=used_j_before[-1] #if there are aligned points before, we include all unaligned points between the current j index and the previous
          starting_pts=list(range(prev_j+1,j0+1))
        else: starting_pts=[j0] 

        if used_j_after: #we do the same for the points after
          next_j=used_j_after[0]
          ending_pts=list(range(j1,next_j+1))
        else: ending_pts=[j1] #

        for j0_ in starting_pts: #now we can add the src and trg phrases to the final phrase list
          for j1_ in ending_pts:
            #print("TRG EXTENDED:", j0_, j1_, trg[j0_:j1_+1])
            trg_phrase=" ".join(trg[j0_:j1_+1])
            all_phrase_pairs.append((src_phrase,trg_phrase))
        # print("used_j_before", used_j_before)
        # print("used_j_after", used_j_after)
        # print("starting_pts", starting_pts)
        # print("ending_pts",ending_pts)
        #print("-----")
  return all_phrase_pairs


def align_extract(src,trg,src_inverted_index,trg_inverted_index, n_iterations=10,max_skip=8, max_phrase_size=10):
  src=["<s>"]+src+["</s>"] #adding start and end sentences
  trg=["<s>"]+trg+["</s>"]
  if len(src)+len(trg)>150: n_iterations=2 #when sentences are very long, do only two iterations
  cur_wt_list=match_sentences_indexes(src,trg,src_inverted_index,trg_inverted_index)
  pts=get_candidate_pts(cur_wt_list,len(src),len(trg))
  pts.sort()
  full_span=((0,0),pts[-1])
  cur_wt_dict=dict(iter(cur_wt_list))
  cur_se_span_dict,cur_ne_span_dict,cur_pt_dict={},{},{}
  for p in pts:
    wt=cur_wt_dict[p]
    cur_se_span_dict[(p,p)]=wt
    cur_ne_span_dict[(p,p)]=wt
    cur_pt_dict[(p,p)]=[p]
  prev_align_wt=0
  for _ in range(n_iterations):
    cur_se_span_dict,cur_ne_span_dict,cur_pt_dict=get_path_rect(cur_se_span_dict,cur_ne_span_dict,cur_pt_dict,True)
    cur_se_span_dict,cur_ne_span_dict,cur_pt_dict=get_path_rect(cur_se_span_dict,cur_ne_span_dict,cur_pt_dict,False)
    cur_align_wt=cur_se_span_dict[full_span]
    cur_align_pts=cur_pt_dict[full_span]
    #print("iteration #", _, "cur_align_wt",cur_align_wt,"cur_align_pts",cur_align_pts)
    if cur_align_wt==prev_align_wt: break
    prev_align_wt=cur_align_wt
  solution=cur_align_pts
  # for i_,j_ in solution:
  #   print(i_,j_,src[i_],trg[j_])
  #cur_solution=get_rect_spans(pts,cur_wt_list,n_iterations,max_skip)
  phrase_pairs=phrase_extraction(src,trg,solution, max_phrase_size)
  return solution,phrase_pairs


In [ ]:
#indexing
import pandas as pd
import numpy as np

batch_size=200000

from itertools import groupby
import unicodedata
def is_punct(token):
  return unicodedata.category(token[0])[0]=="P"

def index_bitext(bitext_fpath,ignore_punc=True):
  fwd_index_src=[]
  fwd_index_trg=[]
  all_src_sentences=[]
  all_trg_sentences=[]
  bitext_fopen=open(bitext_fpath)
  for sent_i,s0_t0 in enumerate(bitext_fopen):
    #print(counter)
    split=s0_t0.strip("\n\r").split("\t")
    if len(split)!=2: continue
    s0,t0=split
    raw_src_toks=[v.lower() for v in tok(s0)]
    raw_trg_toks=[v.lower() for v in tok(t0)]
    #print(raw_trg_toks)
    if ignore_punc:
      raw_src_toks=[v for v in raw_src_toks if not is_punct(v)]
      raw_trg_toks=[v for v in raw_trg_toks if not is_punct(v)]
    
    s_toks=[(v,sent_i,s_i) for s_i,v in enumerate(raw_src_toks)]
    t_toks=[(v,sent_i,t_i) for t_i,v in enumerate(raw_trg_toks)]

    all_src_sentences.append(raw_src_toks)
    all_trg_sentences.append(raw_trg_toks)

    fwd_index_src.extend(s_toks)
    fwd_index_trg.extend(t_toks)

  fwd_index_src.sort()
  fwd_index_trg.sort()
  src_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(fwd_index_src,lambda x:x[0])]
  trg_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(fwd_index_trg,lambda x:x[0])]

  src_inverted=dict(iter(src_grouped))
  trg_inverted=dict(iter(trg_grouped))
  return src_inverted, trg_inverted, all_src_sentences, all_trg_sentences


# test="/"
# print(is_punct(test))

root_dir="uncorpus"
src_file_fpath="UNv1.0.ar-en.en"
#trg_file_fpath="UNv1.0.ar-en-clean2.ar"
trg_file_fpath="UNv1.0.ar-en-clean-tok.ar"
counter=0
src_fopen=open(src_file_fpath,encoding="utf-8")
trg_fopen=open(trg_file_fpath,encoding="utf-8")
fwd_index_src=[]
fwd_index_trg=[]
all_src_sentences=[]
all_trg_sentences=[]
for s0,t0 in zip(src_fopen,trg_fopen):
  #print(counter)
  raw_src_toks=[v.lower() for v in tok(s0)]
  raw_trg_toks=[v.lower() for v in tok(t0)]
  #print(raw_trg_toks)
  all_src_sentences.append(raw_src_toks)
  all_trg_sentences.append(raw_trg_toks)

  s_toks=[(v,counter) for v in raw_src_toks if not is_punct(v)]
  t_toks=[(v,counter) for v in raw_trg_toks if not is_punct(v)]
  fwd_index_src.extend(s_toks)
  fwd_index_trg.extend(t_toks)

  # print(" ".join(s_toks))
  # print(" ".join(t_toks))
  # print("-----")
  if counter>batch_size: 
    break

  counter+=1

fwd_index_src.sort()
fwd_index_trg.sort()
src_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(fwd_index_src,lambda x:x[0])]
trg_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(fwd_index_trg,lambda x:x[0])]

src_inverted=dict(iter(src_grouped))
trg_inverted=dict(iter(trg_grouped))

print("finished indexing %s sentences, now testing on sentences"%batch_size)

src_fopen.close()
trg_fopen.close()

finished indexing 200000 sentences, now testing on sentences


In [ ]:
excluded_src_tokens=["the","a","an","and","of","its","his","her","to","is","are","am","been", "have","has","had", "in", "on", "at", "with", "from","for","that","he","she","it"] #we will need to update this
excluded_trg_tokens=["من","على","في","مع", "إلى","له","تم","إلى","في"]

#so far, all works pretty well - further optimizations would go:
#improve stemming in Arabic and English
#accommodate bigrams and trigrams matching in both src and trg
#improve stop word removal in both src and trg
#improve Arabic tokenization
#prune corner words (which have colinear spans on both horizontal and vertical)

from time import time as tm
test_i=426
test_i=410
test_i=30
test_i=425 #keep for testing
test_i=106279

phrase_dir="phrases"
if not os.path.exists(phrase_dir): os.mkdir(phrase_dir)

subset_size=500
#first_subset_i=0
first_subset_i=292

total_num_of_subsets=400

for subset_i in range(first_subset_i,total_num_of_subsets):
  print("subset_i",subset_i)
  all_src_phrases=[]
  all_trg_phrases=[]
  matching_dict={}
  src_counter,trg_counter={},{}


  start_i=subset_i*subset_size
  end_i=start_i+subset_size
  t0=tm()

  cur_phrase_fname="phrase_test_%s.txt"%start_i
  cur_phrase_fpath=os.path.join(phrase_dir,cur_phrase_fname)
  fopen=open(cur_phrase_fpath,"w")
  for test_i in range(start_i,end_i):
    if test_i%100==0: 
      print(test_i, tm()-t0)
      t0=tm()
    
    src,trg=all_src_sentences[test_i],all_trg_sentences[test_i]
    if len(src)>30: continue #CHECK - add parameter
    #print(test_i, "src", len(src),"trg", len(trg))
    solution_pts,phrase_pairs=align_extract(src,trg,src_inverted,trg_inverted, n_iterations=10,max_skip=8, max_phrase_size=10)
    for src_ph, trg_ph in phrase_pairs:
      if src_ph in all_src_phrases: src_id=all_src_phrases.index(src_ph)
      else:
        src_id=len(all_src_phrases)
        all_src_phrases.append(src_ph)
      src_counter[src_id]=src_counter.get(src_id,0)+1
      if trg_ph in all_trg_phrases: trg_id=all_trg_phrases.index(trg_ph)
      else:
        trg_id=len(all_trg_phrases)
        all_trg_phrases.append(trg_ph)
      trg_counter[trg_id]=trg_counter.get(trg_id,0)+1
      tmp_match_dict=matching_dict.get(src_id,{})
      tmp_match_dict[trg_id]=tmp_match_dict.get(trg_id,0)+1
      matching_dict[src_id]=tmp_match_dict

  src_counter_keys=list(sorted(src_counter.items(),key=lambda x:-x[1]))

  for s_id, s_count_total in src_counter_keys:
    src_phrase=all_src_phrases[s_id]
    trg_corr=matching_dict[s_id]
    trg_corr_counter_keys=list(sorted(trg_corr.items(),key=lambda x:-x[1]))
    matching_list=[]
    for t_id, t_corr_count in trg_corr_counter_keys:
      if t_corr_count<3: continue #CHECK - add parameter
      t_count_total=trg_counter[t_id]
      trg_phrase=all_trg_phrases[t_id]
      numerator=t_corr_count
      denominator=t_count_total+s_count_total-t_corr_count
      ratio=round(float(numerator)/denominator,3)
      matching_list.append((trg_phrase,t_corr_count, ratio))
      line="%s\t%s\t%s\t%s\t%s\t%s\n"%(src_phrase,trg_phrase,ratio,t_corr_count,s_count_total,t_count_total)
      fopen.write(line)
    if not matching_list: continue
  fopen.close()


subset_i 292
146000 4.483112096786499
146100 24.08191204071045
146200 24.26154351234436
146300 38.318477392196655
146400 67.43940377235413
subset_i 293
146500 0.0030090808868408203
146600 22.901004791259766
146700 35.07422924041748
146800 40.7068989276886
146900 61.97652530670166
subset_i 294
147000 0.0031282901763916016
147100 20.173668146133423
147200 28.07275366783142
147300 39.07998323440552
147400 53.57484292984009
subset_i 295
147500 0.002821207046508789
147600 15.186358451843262
147700 40.88905954360962
147800 51.38770890235901
147900 64.60437369346619
subset_i 296
148000 0.002943754196166992
148100 17.09677743911743
148200 34.54376173019409
148300 47.07146334648132
148400 66.78142547607422
subset_i 297
148500 0.0027055740356445312
148600 27.687833786010742
148700 32.6679892539978
148800 64.53559064865112
148900 55.04755449295044
subset_i 298
149000 0.003165721893310547
149100 21.065412998199463
149200 38.7221314907074
149300 40.24664545059204
149400 43.49790143966675
subset_i 2

In [ ]:
matching_list.sort(key=lambda x:-x[-1])
for m in matching_list[:300]:
  #if m[-2]<4: continue
  print(m[-1], "Count:", m[-2])
  print(m[0])
  print(m[1])
  print("----")


0.5 Count: 4
<s> in paragraph
<s> وـ في الـ فقرة
----
0.5 Count: 2
<s> international law commission
<s> لجنة الـ قانون الـ دولي
----
0.5 Count: 2
<s> forty - fifth session
<s> الـ دورة الـ خامسة والأربعون
----
0.5 Count: 2
<s> 3 may - 23
<s> 3 أيار / مايو - 23 تموز
----
0.5 Count: 2
<s> 3 may - 23 july
<s> 3 أيار / مايو - 23 تموز / يوليه
----
0.5 Count: 2
3 may - 23
3 أيار / مايو - 23 تموز
----
0.5 Count: 2
3 may - 23 july
3 أيار / مايو - 23 تموز / يوليه
----
0.5 Count: 2
july
تموز / يوليه
----
0.5 Count: 2
<s> on the question of an international criminal jurisdiction
<s> بـ مسألة إنشاء قضاء جنائي دولي
----
0.5 Count: 2
<s> contents
<s> الـ محتويات
----
0.5 Count: 2
<s> page
<s> الـ صفحة
----
0.5 Count: 2
47 / 33 of 25
47 / 33 الـ مؤرخ 25 تشرين
----
0.5 Count: 2
47 / 33 of 25 november
47 / 33 الـ مؤرخ 25 تشرين الـ ثاني / نوفمبر
----
0.5 Count: 2
november
تشرين الـ ثاني / نوفمبر
----
0.5 Count: 2
draft statute for an international criminal court as a matter
مشروع نظام أساسي لـ محكمة جنا

In [ ]:
#large file iterator - MAIN Part
# import pickle, shelve, os
# from time import time
# from array import array

root_dir="uncorpus"
src_file_locs_fpath=os.path.join(root_dir,"src_locs.txt")
trg_file_locs_fpath=os.path.join(root_dir,"trg_locs.txt")
src_file_fpath="UNv1.0.ar-en.en"
trg_file_fpath="UNv1.0.ar-en-clean2.ar"
print("preparing src locs")
number_file_lines(src_file_fpath,src_file_locs_fpath)
print("preparing trg locs")
number_file_lines(trg_file_fpath,trg_file_locs_fpath)

preparing src locs
preparing trg locs


In [ ]:
min_sent_size=3
max_sent_size=15
limit=2500
sample_size=20

counter2=0

# def match_sentences_indexes(cur_src,cur_trg,src_inverted_index,trg_inverted_index):
#   excluded_src_tokens=["the","a","an","and","of","its","his","her","to","is","are","am"] #we will need to update this
#   excluded_trg_tokens=[]

#   #src_sent=["<s>"]+cur_src+["</s>"] #adding start and end sentences
#   #trg_sent=["<s>"]+cur_trg+["</s>"]
#   src_sent=cur_src
#   trg_sent=cur_trg

#   #array1=np.zeros((len(trg_sent), len(src_sent))) #starting to populate the numpy probability array with zeros
#   wt_list=[]
#   #TODO: optimize the code for using sentence indexes for words, to save calculations for words occuring more than once in the sentence
#   for j_ in range(len(trg_sent)): #now we identify the probability of matching between each pair of src and trg words
#     t_tok=trg_sent[j_]
#     #TODO: do generic steps for exluding stopwords in both source and target words
#     if "ـ" in t_tok: t_indexes=[] #if an Arabic word has tatweel in our tokenization scheme it is a prefix or suffix, so it is excluded from matching
#     else: t_indexes=trg_inverted_index.get(t_tok,[])    #else get the indexes from the inverted index
#     for i_ in range(len(src_sent)):
#       s_tok=src_sent[i_]
#       if s_tok==t_tok and s_tok in ["<s>","</s>"]: ratio=1
#       elif s_tok in excluded_src_tokens: ratio=0
#       else: 
#         s_indexes=src_inverted_index.get(s_tok,[])
#         ratio=idx_match(s_indexes,t_indexes)
      
#       #array1[i_,j_]=ratio #we populate the array with the matching ratio - Maybe not needed, except for visualization, when available
#       wt_list.append(((i_,j_),ratio)) #and also add the coordinates and the ratio to a list
  
#   #print(array1)
#   wt_list.sort(key=lambda x:-x[1]) #we sort the weight list by the probability of each coordinate
#   return wt_list #return wt_list and matching probability list

# def get_candidate_pts(wt_list,src_size,trg_size): #or some dictionary
#   final_list=[]
#   src_side_dict=dict(iter([(i,[0]*trg_size) for i in range(src_size)] )) #dict with keys corresponding to src sent word indexes, and values are the probability of each target word index
#   trg_side_dict=dict(iter([(i,[0]*src_size) for i in range(trg_size)] )) #dict with keys corresponding to trg

#   #trg_side_dict=dict(iter([(i,[0]*trg_size) for i in range(src_size)] )) #dict with keys corresponding to src sent word indexes, and values are the probability of each target word index
#   #src_side_dict=dict(iter([(i,[0]*src_size) for i in range(trg_size)] )) #dict with keys corresponding to trg


#   #we collect the coordinates with the highest probability across the matrix, with unique x,y/i,j coordinates
#   used_i,used_j=[],[] #if an index (i or j) is used once in a coordinate, it is excluded 
#   #final_list=[]
#   for ij,wt in wt_list: #while we go in a descending order in all the coordinates with their weights
#     if wt==0: continue
#     i0,j0=ij
#     src_side_dict[i0][j0]=wt
#     trg_side_dict[j0][i0]=wt

#     #if i0==0 and j0!=0: continue
#     #if i0!=0 and j0==0: continue
#     if i0 in used_i or j0 in used_j: continue
#     #if i0 in used_i and j0 in used_j: continue
#     if not (i0,j0) in final_list: 
#       final_list.append((i0,j0)) #these unique coordinates are also added to the final candidate points
#       #array1[i0,j0]+=25
#     used_i.append(i0)
#     used_j.append(j0)

#     for i_ in range(src_size): #get the maximum coordinates corresponding to each source word
#       corr=src_side_dict[i_]
#       max_corr=max(corr)
#       if max_corr==0: continue
#       max_j_indexes=[k0 for k0,v in enumerate(corr) if v==max_corr]
#       for j_ in max_j_indexes:
#         final_list.append((i_,j_))

#     for j_ in range(trg_size): #and the maximum corresponding to each target word
#       corr=trg_side_dict[j_]
#       max_corr=max(corr)
#       if max_corr==0: continue
#       max_i_indexes=[k0 for k0,v in enumerate(corr) if v==max_corr]
#       for i_ in max_i_indexes:
#         final_list.append((i_,j_)) 
#   final_list=list(set(final_list)) #now we have the final list of possible points, that we can process with dynamic programming
#   final_list.sort()
#   return final_list #return candidate points, which are maximum points in rows and columns and overall




# def align_extract(src,trg,src_inverted_index,trg_inverted_index, n_iterations=10,max_skip=8, max_phrase_size=10):
#   src=["<s>"]+src+["</s>"] #adding start and end sentences
#   trg=["<s>"]+trg+["</s>"]
#   cur_wt_list=match_sentences_indexes(src,trg,src_inverted_index,trg_inverted_index)
#   pts=get_candidate_pts(cur_wt_list,len(src),len(trg))
#   cur_solution=get_rect_spans(pts,cur_wt_list,n_iterations,max_skip)
#   phrase_pairs=phrase_extraction(src,trg,cur_solution)
#   return phrase_pairs




class align:
  def __init__(self,pts, wt_list,out_file=None):
    self.solution=[]
    self.pts=sorted(pts)
    self.last_pt=self.pts[-1]
    self.align_span=((0,0),self.last_pt)
    self.composite_span_dict={}
    self.se_spans_dict={}
    self.ne_spans_dict={}

    self.se=True #going into the diagonal direction
    self.out_file=out_file
      
      
    
    self.wt_dict=dict(iter(wt_list))
    for p in self.pts:
      cur_wt=self.wt_dict.get(p,0)
      self.se_spans_dict[(p,p)]=cur_wt 
      self.ne_spans_dict[(p,p)]=cur_wt
      if self.out_file: self.out_file.write("%s\t%s\n"%(p, cur_wt))
    if self.out_file: self.out_file.write("\n=====\n")

  def get_opt_paths(self,se=True): #Main alignment function
    prev_span_dict={} #prev_span_se_dict - initialize dictionaries
    prev_wt_dict={} #prev_wt_se_dict

    if se: 
      cur_span_dict=self.se_spans_dict
      list_spans=sorted(list(cur_span_dict.keys()),key=lambda x:x[0][0]) #sort the keys of the span dict towards SE
    else: 
      cur_span_dict=self.ne_spans_dict
      list_spans=sorted(list(cur_span_dict.keys()),key=lambda x:(x[0][0],-x[0][1]) ) #sort the keys towards NE

    for li,cur_span in enumerate(list_spans): #going through each of the spans
      cur_span_wt=cur_span_dict.get(cur_span,0) #actual weight of the current span
      cur_span_path_wt=prev_wt_dict.get(cur_span,cur_span_wt) #cumulative weight of the path up to current span
      prev_wt_dict[cur_span]=cur_span_path_wt #we update the weight in the path/prev dict 
      cur_pt1,cur_pt2=cur_span
      cur_x1,cur_y1=cur_pt1
      cur_x2,cur_y2=cur_pt2
      if se: next_spans=[(v_p1,v_p2) for v_p1,v_p2 in list_spans[li+1:] if v_p1[1]>cur_y2] #identify all points towards SE from the current point
      else: next_spans=[(v_p1,v_p2) for v_p1,v_p2 in list_spans[li+1:] if v_p1[1]<cur_y2] #identify all points towards NE from the current point

      #print("SE", se, "cur_span",cur_span,"next_spans", len(next_spans),next_spans[:5])

      for next_span in next_spans:    #iterate over each of these SE points
        if next_span==cur_span: continue
        next_pt1,next_pt2=next_span
        next_x1,next_y1=next_pt1
        next_x2,next_y2=next_pt2
        #   specify admissibility conditions, else skip
        if cur_x1==cur_x2==next_x1==next_x2: pass #all spans are on the same column (have the same X)
        elif cur_y1==cur_y2==next_y1==next_y2: pass #all spans are on the same row (have the same Y)
        elif cur_pt1==cur_pt2 and next_pt1==next_pt2: pass #if both spans are for single points  
        elif next_x1>cur_x2: pass #if the start X of the second is after the end X of the first
        else: continue

        #now let's merge the two spans
        next_span_wt=cur_span_dict.get(next_span,0) #get the actua weight of the next SE span
        if cur_x1==cur_x2==next_x1==next_x2: 
          ###cur_span_wt
          combined_span_wt=max(cur_span_path_wt,next_span_wt) #for colinear spans, we use only the maximum span weight
        elif cur_y1==cur_y2==next_y1==next_y2: 
          ###cur_span_wt 
          combined_span_wt=max(cur_span_path_wt,next_span_wt)
        else: combined_span_wt=cur_span_path_wt+next_span_wt #combine weight so far (path/prev weight) of the current span with the weight of next span

        if self.out_file: self.out_file.write("%s\t%s\t%s\t%s - combined wt: %s\n"%(cur_span, round(cur_span_path_wt,3) , next_span, round(next_span_wt,3) ,round(combined_span_wt,3) ))
        
        found_combined_span_path_wt=prev_wt_dict.get(next_span,0) #check if there is a larger weight for the next span in the path/prev weight dictionary
        #print(cur_span,next_span,cur_span_path_wt,next_span_wt, "combined:",combined_span_wt, "found:", found_combined_span_path_wt )
        if combined_span_wt>found_combined_span_path_wt: #if the combined weight of the next span and path so far for current is greater than the path so far for next span
          prev_wt_dict[next_span]=combined_span_wt #we update the path of the weight so far for the next span
          prev_span_dict[next_span]=cur_span #and also update the current span as the previous point in the optimum path
          cur_span_dict[(cur_pt1,next_pt2)]=combined_span_wt ##<<<<<<<<<< checking

    if self.out_file: self.out_file.write("Paths\n\n")
    if self.out_file: 
      if se: self.out_file.write("Going SE\n\n")
      else: self.out_file.write("Going NE\n\n")

    #for each span, we identify the optimum path leading to it
    all_paths=[] #now we get all the paths from each span till the beginning
    for k,v in prev_span_dict.items(): 
      found=v
      path=[k]
      if found: path.append(found)
      while found:
        found=prev_span_dict.get(found)
        if found: path.append(found)
      all_paths.append(path)
      if self.out_file: self.out_file.write("path found: %s \n"%path)

    self.span_pt_dict={} #all the points in a given span
    all_cur_spans=[]
    for pth in all_paths: #now we will rectangle each two points on each path into new spans (both SE and NE), and get the highest weight for each span
      pth.reverse()
      
      wts=[cur_span_dict[v] for v in pth]
      print("SE", se, "PATH:",pth, "Weights:", sum(wts))
      for pi0 in range(len(pth)):
        for pi1 in range(pi0+1,len(pth)):
          cur_spans=pth[pi0:pi1+1]
          if cur_spans in all_cur_spans: continue
          all_cur_spans.append(cur_spans)

          span1,span2=pth[pi0],pth[pi1]

          #full_span=(span1[0],span2[1]) 
          mixed_spans=span1+span2 #calculate the window of the two spans
          mixed_spans_xs=[v[0] for v in mixed_spans]
          mixed_spans_ys=[v[1] for v in mixed_spans]
          min_x,max_x=min(mixed_spans_xs),max(mixed_spans_xs)
          min_y,max_y=min(mixed_spans_ys),max(mixed_spans_ys)

          full_se_span=((min_x,min_y),(max_x,max_y))
          full_ne_span=((min_x,max_y),(max_x,min_y))
          cur_wts=wts[pi0:pi1+1]
          #cur_pts=list(set([v[0] for v in cur_spans if v[0]==v[1]]))
          
          cur_pts=[]
          for cs in cur_spans:
            if cs[0]==cs[1]: 
              cur_pts.append(cs[0])
            else:
              found_pts=self.span_pt_dict.get(cs,[])
              cur_pts.extend(found_pts)
          if self.out_file: self.out_file.write("full_SE_span: %s - full_NE_span: %s - cur_spans: %s \n"%(full_se_span, full_ne_span, cur_spans))
          

          #print("full_se_span", full_se_span)
          # if se==False:
          #   print("full_ne_span",full_ne_span,"cur_spans",cur_spans)
            
          #   print("Points:", cur_pts)
          #   print("---------")
            

            #print("inner span", cs)

          
          sum_wt=sum(cur_wts)
          found_se_span_wt=self.se_spans_dict.get(full_se_span,0)

          if sum_wt>found_se_span_wt: 
            self.se_spans_dict[full_se_span]=sum_wt
            self.span_pt_dict[full_se_span]=cur_pts
            self.composite_span_dict[full_se_span]=cur_spans

            #print("full_se_span",full_se_span, "sum_wt",sum_wt, "cur_spans",cur_spans )
          found_ne_span_wt=self.ne_spans_dict.get(full_ne_span,0)
          if sum_wt>found_ne_span_wt: 
            self.ne_spans_dict[full_ne_span]=sum_wt
            self.span_pt_dict[full_ne_span]=cur_pts
            self.composite_span_dict[full_se_span]=cur_spans


  def iterate(self,n_iter=5): 
    prev_wt=0   
    for _ in range(n_iter):
      if self.out_file: self.out_file.write("iteration # %s"%_)
      print("iteration # ",_)

      self.get_opt_paths(se=True)
      print("Full span >>>>", self.se_spans_dict.get(self.align_span), self.composite_span_dict.get(self.align_span))
      self.get_opt_paths(se=False)
      print("Full span >>>>", self.se_spans_dict.get(self.align_span), self.composite_span_dict.get(self.align_span))
      for a in self.composite_span_dict.get(self.align_span,[]):
        print("$$$", a)
      print("----")
      cur_wt=self.se_spans_dict.get(self.align_span)
      final_pts=self.span_pt_dict.get(self.align_span)
      print("final_pts",final_pts)
      for a in list(self.span_pt_dict.keys())[:10]:
        print(a, self.span_pt_dict[a])
      print("=========")
      
      if cur_wt==prev_wt: break
      prev_wt=cur_wt
    comp=self.composite_span_dict.get(self.align_span)
    all_spans=[]
    new_spans=list(comp)
    while new_spans:
      new_spans_copy=[]
      for span0 in new_spans:
        
        sub_spans=self.composite_span_dict.get(span0,[])
        if sub_spans==[]: 
          all_spans.append(span0)
          #print("current span:", span0)
        else:
          print("current span:", span0, "sub_spans", sub_spans)

          new_spans_copy.extend(sub_spans)
      new_spans=list(new_spans_copy)
    self.solution=[v[0] for v in sorted(all_spans)]
    return self.solution    
        
#===============
n_iterations=2

test_i=426
test_i=410
test_i=30
test_i=425 #keep for testing
#test_i=426
src,trg=all_src_sentences[test_i],all_trg_sentences[test_i]
print(len(src), len(trg))
src=["<s>"]+src+["</s>"] #adding start and end sentences
trg=["<s>"]+trg+["</s>"]
cur_wt_list=match_sentences_indexes(src,trg,src_inverted,trg_inverted)
pts=get_candidate_pts(cur_wt_list,len(src),len(trg))

cur_wt_dict=dict(iter(cur_wt_list))
#print(len(pts), pts[:10])
out_fopen=open("test.txt","w")
al_obj=align(pts,cur_wt_list,out_fopen)

solution=al_obj.iterate(n_iterations)
out_fopen.close()

phrase_pairs=phrase_extraction(src,trg,solution)
# for sol in solution:
#   print(sol)

print(list(enumerate(src)))
print(list(enumerate(trg)))
for i0,j0 in solution:
  print(">>>", i0,j0,src[i0],trg[j0])

import numpy as np

array1=np.zeros((len(src), len(trg))) #starting to populate the numpy probability array with zeros
for i_ in range(len(src)): #now we identify the probability of matching between each pair of src and trg words
  s_tok=src[i_]
  for j_ in range(len(trg)):
    t_tok=trg[j_]
    wt=round(cur_wt_dict[(i_,j_)],3)
    if (i_,j_) in pts: wt+=1
    array1[i_,j_]= wt #we populate the array with the matching ratio

array1=array1.transpose()
# for fp in phrase_pairs:
#   print(fp[0])
#   print(fp[1])
#   print("----")
#df = pd.DataFrame(array1, index=trg_sent, columns=src_sent)
df = pd.DataFrame(array1, index=trg, columns=src)

display(df.round(3))

from pandas import *
df.to_html('filename.html')

# import matplotlib.pyplot as plt
# import pandas as pd
# #from pandas.table.plotting import table # EDIT: see deprecation warnings below
# from pandas.plotting import table 

# ax = plt.subplot(111, frame_on=False) # no visible frame
# ax.xaxis.set_visible(False)  # hide the x axis
# ax.yaxis.set_visible(False)  # hide the y axis

# table(ax, df)  # where df is your data frame

# plt.savefig('mytable.png')
# plt.show()


In [ ]:
list_src_phrases=[]
list_trg_phrases=[]
src_phrase_counter={}
trg_phrase_counter={}
matching_dict={}

for test_i in range(500):
  src,trg=all_src_sentences[test_i],all_trg_sentences[test_i]
  if len(src)>40: continue
  print(test_i, "src:", len(src), "trg:", len(trg))
  cur_phrase_pairs=align_extract(src,trg,src_inverted,trg_inverted,10,10)
  print("pairs extracted:", len(cur_phrase_pairs))

  for src_p,trg_p in cur_phrase_pairs:
    
    if src_p in list_src_phrases: src_id=list_src_phrases.index(src_p)
    else: 
      src_id=len(list_src_phrases)
      list_src_phrases.append(src_p)
    if trg_p in list_trg_phrases: trg_id=list_trg_phrases.index(trg_p)
    else: 
      trg_id=len(list_trg_phrases)
      list_trg_phrases.append(trg_p)
    tmp_dict=matching_dict.get(src_id,{})
    tmp_dict[trg_id]=tmp_dict.get(trg_id,0)+1
    
    matching_dict[src_id]=tmp_dict
    src_phrase_counter[src_id]=src_phrase_counter.get(src_id,0)+1
    trg_phrase_counter[trg_id]=trg_phrase_counter.get(trg_id,0)+1


0 src: 5 trg: 5
iteration # 0 number of keys: 6
iteration # 1 number of keys: 19
iteration # 2 number of keys: 19
pairs extracted: 79
1 src: 3 trg: 5
iteration # 0 number of keys: 5
iteration # 1 number of keys: 12
iteration # 2 number of keys: 12
pairs extracted: 69
2 src: 4 trg: 5
iteration # 0 number of keys: 6
iteration # 1 number of keys: 16
iteration # 2 number of keys: 16
pairs extracted: 71
3 src: 6 trg: 10
iteration # 0 number of keys: 8
iteration # 1 number of keys: 34
iteration # 2 number of keys: 35
iteration # 3 number of keys: 35
pairs extracted: 136
4 src: 10 trg: 11
iteration # 0 number of keys: 14
iteration # 1 number of keys: 76
iteration # 2 number of keys: 80
iteration # 3 number of keys: 80
pairs extracted: 276
5 src: 8 trg: 6
iteration # 0 number of keys: 13
iteration # 1 number of keys: 50
iteration # 2 number of keys: 50
pairs extracted: 151
6 src: 1 trg: 2
iteration # 0 number of keys: 3
iteration # 1 number of keys: 6
iteration # 2 number of keys: 6
pairs extr

In [ ]:
md_keys=list(matching_dict.keys())
print(len(md_keys))
for src_id in md_keys[35000:36000]:
  corr=matching_dict[src_id].items()
  #valid_corr=[(v,w) for v,w in corr if w>1]
  valid_corr=[(v,w) for v,w in corr]
  valid_corr.sort(key=lambda x:-x[1])
  if not valid_corr: continue
  src_phrase=list_src_phrases[src_id]
  phrase_count=src_phrase_counter[src_id]
  print(src_phrase)
  print("count:", phrase_count)
  print("corr:", valid_corr)
  for trg_id,t_count in valid_corr[:3]:
    overall_trg_phrase_count=trg_phrase_counter[trg_id]
    print(list_trg_phrases[trg_id], ">>> count:",t_count)
    
  print("----")

Streaming output truncated to the last 5000 lines.
----
aggression is a different
count: 9
corr: [(55784, 1), (55785, 1), (55786, 1), (55787, 1), (55788, 1), (55789, 1), (55790, 1), (55791, 1), (55792, 1)]
حالة الـ عدوان >>> count: 1
حالة الـ عدوان . >>> count: 1
حالة الـ عدوان . </s> >>> count: 1
----
aggression is a different case
count: 9
corr: [(55784, 1), (55785, 1), (55786, 1), (55787, 1), (55788, 1), (55789, 1), (55790, 1), (55791, 1), (55792, 1)]
حالة الـ عدوان >>> count: 1
حالة الـ عدوان . >>> count: 1
حالة الـ عدوان . </s> >>> count: 1
----
aggression is a different case .
count: 9
corr: [(55784, 1), (55785, 1), (55786, 1), (55787, 1), (55788, 1), (55789, 1), (55790, 1), (55791, 1), (55792, 1)]
حالة الـ عدوان >>> count: 1
حالة الـ عدوان . >>> count: 1
حالة الـ عدوان . </s> >>> count: 1
----
aggression is a different case . </s>
count: 3
corr: [(55786, 1), (55789, 1), (55792, 1)]
حالة الـ عدوان . </s> >>> count: 1
الـ عدوان . </s> >>> count: 1
عدوان . </s> >>> count: 1
----
is

In [ ]:
#phrase extraction step
src=['<s>', 'promoted', 'to', 'first', 'secretary', 'of', 'embassy', ',', 'december', '1990', '.', '</s>']
trg=['<s>', 'رقي', 'إلى', 'سكرتير', 'سفارة', 'أول', '،', 'في', 'كانون', 'الـ', 'أول', '/', 'ديسمبر', '1990', '.', '</s>']
solution=[(0, 0), (1, 1), (2, 2), (4, 6), (5, 3), (7, 5), (10, 4), (12, 8), (13, 9), (15, 11)]


# src=['<s>', 'since', '1989', ',', 'representative', 'of', 'argentina', 'to', 'the', 'fifth', 'committee', '.', '</s>']
# trg=['<s>', 'صار', 'ممثلا', 'للأرجنتين', 'في', 'الـ', 'لجنة', 'الـ', 'خامسة', 'منذ', 'عام', '1989', '.', '</s>']
# solution=[(0, 0), (1, 8), (3, 6), (4, 7), (6, 10), (8, 9), (9, 1), (10, 5), (11, 2), (13, 12)]

src=['<s>', 'member', 'of', 'the', 'argentine', 'delegation', 'to', 'the', 'second', 'committee', 'in', '1985', 'and', '1988', '.', '</s>']
trg=['<s>', 'وـ', 'عضو', 'الـ', 'وفد', 'الـ', 'أرجنتيني', 'في', 'الـ', 'لجنة', 'الـ', 'ثانية', '،', 'عامي', '1985', 'و', '1988', '.', '</s>']
solution=[(0, 0), (2, 1), (4, 5), (6, 4), (7, 10), (9, 9), (11, 8), (13, 6), (14, 11), (15, 12), (16, 13), (18, 15)]

src=['<s>', 'protocols', 'were', 'signed', 'by', 'the', 'presidents', 'of', 'both', 'countries', 'in', 'july', '1986', '.', '</s>']
trg=['<s>', 'وـ', 'قد', 'وقع', 'الـ', 'بروتوكولات', 'رئيسا', 'الـ', 'بلدين', 'في', 'تموز', '/', 'يوليه', 'عام', '1986', '.', '</s>']
solution=[(0, 0), (2, 2), (3, 3), (5, 1), (6, 6), (8, 9), (9, 10), (10, 11), (12, 8), (13, 4), (14, 12), (16, 14)]

src=['<s>', 'member', 'of', 'the', 'coordinating', 'group', 'of', 'the', 'argentine', '-', 'brazilian', 'integration', 'process', '.', '</s>']
trg=['<s>', 'عضو', 'فريق', 'تنسيق', 'عملية', 'تكامل', 'الـ', 'أرجنتين', 'والبرازيل', '.', '</s>']
solution=[(0, 0), (1, 1), (2, 5), (3, 4), (4, 12), (5, 11), (7, 8), (8, 10), (10, 14)]

# src=['<s>', 'the', 'candidate', "'s", 'curriculum', 'vitae', 'appears', 'in', 'the', 'annex', 'to', 'the', 'present', 'note', '.', '</s>']
# trg=['<s>', 'وـ', 'ترد', 'في', 'مرفق', 'هذه', 'الـ', 'مذكرة', 'سيرة', 'الـ', 'مرشح', 'الـ', 'شخصية', '.', '</s>']
# solution=[(0, 0), (2, 12), (3, 7), (4, 9), (5, 10), (7, 13), (8, 5), (10, 2), (12, 4), (14, 15)]


phrase_size_limit=10


cur_pairs=phrase_extraction(src,trg,solution,8)
for cp in cur_pairs:
  print(cp) 

('<s> member', '<s> عضو')
('<s> member', '<s> عضو فريق')
('<s> member of', '<s> عضو')
('<s> member of', '<s> عضو فريق')
('<s> member of the', '<s> عضو')
('<s> member of the', '<s> عضو فريق')
('<s> member of the coordinating group', '<s> عضو فريق تنسيق')
('<s> member of the coordinating group', '<s> عضو فريق تنسيق عملية')
('<s> member of the coordinating group of', '<s> عضو فريق تنسيق')
('<s> member of the coordinating group of', '<s> عضو فريق تنسيق عملية')
('<s> member of the coordinating group of the', '<s> عضو فريق تنسيق')
('<s> member of the coordinating group of the', '<s> عضو فريق تنسيق عملية')
('member', 'عضو')
('member', 'عضو فريق')
('member of', 'عضو')
('member of', 'عضو فريق')
('member of the', 'عضو')
('member of the', 'عضو فريق')
('member of the coordinating group', 'عضو فريق تنسيق')
('member of the coordinating group', 'عضو فريق تنسيق عملية')
('member of the coordinating group of', 'عضو فريق تنسيق')
('member of the coordinating group of', 'عضو فريق تنسيق عملية')
('member of 

In [ ]:

# print(len(src))
# print(len(trg))
# for s,t in solution:
#   print(s,t, src[s], trg[t])
# BP=[]
# for e_start in range(1, len(src)):
#   for e_end in range(e_start, len(src)):
#     if e_end-e_start>phrase_size_limit: break
#     print(e_start,e_end,  src[e_start:e_end])
#     f_start,f_end = len(trg), 0
#     for e0,f0 in solution:
#       if e_start <= e0 and e0<=e_end:
#         f_start = min(f0,f_start)
#         f_end= max(f0, f_end)
#     print(f_start,f_end,  trg[f_start:f_end])
#     print("-----")
#     extracted=extract(f_start,f_end,e_start,e_end)
#     # BP.extend(extracted)


# def extract(f_start0,f_end0,e_start0,e_end0):
#   if f_end0==0: return []
#   for e0,f0 in solution:
#     if e0<e_start0 or e0>e_end0: return []
#   E=[]
#   f_s = f_start0
#   used_j_before=[v for v in used_j if v<f_start0]
#   used_j_after=[v for v in used_j if v>f_end0]
#   print("f_start0,f_end0,e_start0,e_end0", f_start0,f_end0,e_start0,e_end0)

  # while True:
  #   pass


"""
Input: word alignment A for sentence pair (e,f)
Output: set of phrase pairs BP
1: for e start = 1 ... length(e) do
2: for eend = estart ... length(e) do
3: // find the minimally matching foreign phrase
4: (fstart, fend) = ( length(f), 0 )
5: for all (e,f) ∈ A do
6: if estart ≤ e ≤ eend then
7: fstart = min( f, fstart )
8: fend = max( f, fend )
9: end if
10: end for
11: add extract(fstart, fend, estart, eend) to set BP
12: end for
13: end for
function extract(fstart, fend, estart, eend)
1: return { } if fend == 0 // check if at least one alignment point
2: // check if alignment points violate consistency
3: for all (e,f) ∈ A do
4: return { } if e < estart or e > eend
5: end for
6: // add pharse pairs (incl. additional unaligned f)
7: E = { }
8: fs = fstart
9: repeat
10: fe = fend
11: repeat
12: add phrase pair (estart .. eend, fs .. fe) to set E
13: fe ++
14: until fe aligned
15: fs −−
16: until fs aligned
17: return E
Figure 5.5 Phrase extraction algorithm
"""

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]
df.style.apply(highlight_max)

,,has,prosecuted,criminal,cases,as,deputy,public,prosecutor,.,
,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
وعمل,0.000000,0.000667,0.020408,0.001642,0.001427,0.001069,1.024096,0.001366,1.022989,0.000000,0.000000
نائبا,0.000000,0.000167,1.038462,0.001706,0.001475,0.000334,1.027972,0.000693,0.015385,0.000000,0.000000
للمدعي,0.000000,0.000168,3.066667,0.001739,0.001499,0.000268,1.030303,0.001399,1.037736,0.000000,0.000000
العام,0.000000,0.037031,0.000128,0.004327,0.004757,0.053354,0.003804,3.050399,0.003715,0.000000,0.000000
في,0.000000,1.045179,0.000102,0.004062,0.006035,3.092461,0.000662,0.010580,0.000407,0.000000,0.000000
قضايا,0.000000,0.004360,0.003774,0.013497,3.034949,0.004562,0.005208,0.003580,0.003289,0.000000,0.000000
جنائية,0.000000,0.000829,0.012195,3.102916,0.006849,0.001535,0.004950,0.000668,0.008264,0.000000,0.000000
.,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000


In [ ]:

import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

s = df.style.background_gradient(cmap=cm)
s

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,,has,prosecuted,criminal,cases,as,deputy,public,prosecutor,.,
,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
وعمل,0.000000,0.000667,0.020408,0.001642,0.001427,0.001069,1.024096,0.001366,1.022989,0.000000,0.000000
نائبا,0.000000,0.000167,1.038462,0.001706,0.001475,0.000334,1.027972,0.000693,0.015385,0.000000,0.000000
للمدعي,0.000000,0.000168,3.066667,0.001739,0.001499,0.000268,1.030303,0.001399,1.037736,0.000000,0.000000
العام,0.000000,0.037031,0.000128,0.004327,0.004757,0.053354,0.003804,3.050399,0.003715,0.000000,0.000000
في,0.000000,1.045179,0.000102,0.004062,0.006035,3.092461,0.000662,0.010580,0.000407,0.000000,0.000000
قضايا,0.000000,0.004360,0.003774,0.013497,3.034949,0.004562,0.005208,0.003580,0.003289,0.000000,0.000000
جنائية,0.000000,0.000829,0.012195,3.102916,0.006849,0.001535,0.004950,0.000668,0.008264,0.000000,0.000000
.,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000


In [ ]:
#now processing sentences using the indexes
min_sent_size=3
max_sent_size=15
limit=2500
sample_size=20

counter2=0

for src_sent, trg_sent in zip(all_src_sentences,all_trg_sentences): #processing pairs of tokenized sentences
  #if counter2>limit or counter2<limit-200: continue
  if counter2<limit-sample_size: #just pick a sample of the sentences
    counter2+=1
    continue
  if counter2>limit: break
  #if counter2<limit-300: continue
  if len(src_sent)<min_sent_size: continue
  if len(src_sent)>max_sent_size: continue
  if src_sent[-1]!=".": continue #only choose sentences and not headings and table items without full stop at the end

  print(src_sent)
  print(trg_sent)
  src_sent=["<s>"]+src_sent+["</s>"] #adding start and end sentences
  trg_sent=["<s>"]+trg_sent+["</s>"]
  array1=np.zeros((len(trg_sent), len(src_sent))) #starting to populate the numpy probability array with zeros
  wt_list=[]
  for i_ in range(len(trg_sent)): #now we identify the probability of matching between each pair of src and trg words
    t_tok=trg_sent[i_]
    if "ـ" in t_tok: t_indexes=[] #if an Arabic word has tatweel in our tokenization scheme it is a prefix or suffix, so it is excluded from matching
    else: t_indexes=trg_inverted.get(t_tok,[])    #else get the indexes from the inverted index
    for j_ in range(len(src_sent)):
      s_tok=src_sent[j_]
      if s_tok in ["the","of","and","its","his","her", "a","an"]: s_indexes=[]
      else: s_indexes=src_inverted.get(s_tok,[])
      if s_tok==t_tok and s_tok in ["<s>","</s>"]: ratio=1
      else: ratio=idx_match(s_indexes,t_indexes)
      
      array1[i_,j_]=ratio #we populate the array with the matching ratio
      wt_list.append(((i_,j_),ratio)) #and also add the coordinates and the ratio to a list
  
  #print(array1)
  wt_list.sort(key=lambda x:-x[1]) #we sort the weight list by the probability of each coordinate

  final_list=[]

  max_index_col = np.argmax(array1, axis=0) #collect the coordinates of pairs of coordinates with highest probability in each column
  print(max_index_col)
  for j,c in enumerate(max_index_col):
    #if j==0 or j==len(max_index_col)-1: continue
    if j==0: continue
    if not (c,j) in final_list: 
      final_list.append((c,j))
      #array1[c,j]+=5

  max_index_row = np.argmax(array1, axis=1) #and we do so also for each row
  for i,r in enumerate(max_index_row):
    #if i==0 or i==len(max_index_row)-1: continue
    if i==0: continue
    if not (i,r) in final_list: 
      final_list.append((i,r))
      #array1[i,r]+=10
  print(max_index_row)


  #and we also collect the coordinates with the highest probability across the matrix, with unique x,y/i,j coordinates
  used_i,used_j=[],[] #if an index (i or j) is used once in a coordinate, it is excluded 
  #final_list=[]
  for ij,wt in wt_list: #while we go in a descending order in all the coordinates with their weights
    if wt==0: continue
    i0,j0=ij
    #if i0==0 and j0!=0: continue
    #if i0!=0 and j0==0: continue
    if i0 in used_i or j0 in used_j: continue
    #if i0 in used_i and j0 in used_j: continue
    if not (i0,j0) in final_list: 
      final_list.append((i0,j0)) #these unique coordinates are also added to the final candidate points
      #array1[i0,j0]+=25
    used_i.append(i0)
    used_j.append(j0)
    
  final_list=list(set(final_list)) #now we have the final list of possible points, that we can process with dynamic programming
  final_list.sort()
  
  #a span is the rectangle between two points [(0,0),(5,4)]
  span_dict={} #it has the weight of each span, updates with each iteration
  pt_dict={} #it has the points within this span, also updated with each iteration
  for i1,j1 in final_list: #we initilize the span and pt dictionaries with the points in the current candidate list
    if i1==0 and j1!=0: continue
    if i1!=0 and j1==0: continue
    cur_span=((i1,j1),(i1,j1))
    span_dict[cur_span]=array1[i1,j1]
    pt_dict[cur_span]=[(i1,j1)]
    #print(i1,j1)
    array1[i1,j1]+=1 #just to keep track of the candidate points when visualized in the matrix plot
  print("-----")
  full_span=((0,0),final_list[-1]) #this is the span of the full matrix
  pts_combinations=[]
  for _ in range(10): #we do a number of iterations till the alignment converges across the matrix
    print("iteration #",_)

    keys=sorted(span_dict.keys()) #check what spans we have so far
    #print("keys",keys)
    for ki,cur_span in enumerate(keys): #start iterating over spans, merge them to create new spans
      cur_wt=span_dict[cur_span]
      cur_pts=pt_dict[cur_span]
      cur_pt0,cur_pt1=cur_span
      cur_x0,cur_y0=cur_pt0
      cur_x1,cur_y1=cur_pt1
      for next_span in keys[ki+1:]: #checking the surrounding spans
        if cur_span[0][0]==next_span[0][0]: continue
        next_pt0,next_pt1=next_span
        next_x0,next_y0=next_pt0
        next_x1,next_y1=next_pt1
        if next_x0<=cur_x1: continue #spans expand to the right downwards
        if next_y0>=cur_y0 and next_y0<=cur_y1: continue
        if next_y1>=cur_y0 and next_y1<=cur_y1: continue
        if next_y0<=cur_y0 and next_y1>=cur_y1: continue


        next_wt=span_dict[next_span] #we get the weight and point info of the next span to be merged
        next_pts=pt_dict[next_span]
        combined_pts=sorted(cur_pts+next_pts)
        if combined_pts in pts_combinations: continue #when combining pts of cur and next spans, we ignore them if this combination was used before

        pts_combinations.append(combined_pts)
        x0=cur_span[0][0] #now constructing the new span
        x1=next_span[-1][0]
        y0=min(cur_span[0][1],next_span[0][1])
        y1=max(cur_span[-1][1],next_span[-1][1])
        new_span=((x0,y0),(x1,y1))
        new_wt=cur_wt+next_wt #and also the new weight
        new_pts=cur_pts+next_pts #and also the points in this span
        found_wt=span_dict.get(new_span,0) #but we also check if there is a weight for this span from previous iterations

        if new_wt> found_wt: #if the new weight of the merged span is larger than the found weight, we start updating the span_dict and pt_dict
          #print("found_wt", found_wt)
          span_dict[new_span]=new_wt
          pt_dict[new_span]=combined_pts
          #print("new_span", new_span, "combined_pts",combined_pts, "new_wt",new_wt)
          # if new_span==full_span:
          #   pass
            #print("cur_span", cur_span, "next_span",next_span, "new_span", new_span, "new_wt",new_wt,"combined_pts",combined_pts)
            #print("x0,x1",x0, x1, "y0,y1", y0, y1)
            #print("---------")


  solution=pt_dict[full_span] #and the solution is the points that are in the pt_dict for the full span
  print(src_sent)
  print(trg_sent)
  print(solution)
  print("------")


  df = pd.DataFrame(array1, index=trg_sent, columns=src_sent)

  display(df.round(3))
  counter2+=1


['*', 'the', 'text', 'is', 'being', 'distributed', 'in', 'its', 'original', 'version', 'only', '.']
['*', 'يعمم', 'الـ', 'نص', 'في', 'نسخته', 'الـ', 'أصلية', 'فقط', '.']
[ 0  0  0  4  5  5  9  5  0  8  4  9  0 11]
[ 0  0 10  0  3  7 10  0  9 11  0 13]
-----
iteration # 0
iteration # 1
iteration # 2
iteration # 3
iteration # 4
iteration # 5
iteration # 6
iteration # 7
iteration # 8
iteration # 9
['<s>', '*', 'the', 'text', 'is', 'being', 'distributed', 'in', 'its', 'original', 'version', 'only', '.', '</s>']
['<s>', '*', 'يعمم', 'الـ', 'نص', 'في', 'نسخته', 'الـ', 'أصلية', 'فقط', '.', '</s>']
[(0, 0), (2, 6), (4, 3), (5, 7), (6, 10), (8, 9), (9, 11), (11, 13)]
------


,<s>,*,the,text,is,being,distributed,in,its,original,version,only,.,</s>
<s>,2.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.0
*,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.0
يعمم,0.0,0.0,0.0,0.006,0.000,0.002,1.005,0.000,0.0,0.003,1.010,0.001,0.0,0.0
الـ,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.0
نص,0.0,0.0,0.0,1.441,0.005,0.002,0.008,0.004,0.0,0.002,1.012,0.003,0.0,0.0
في,0.0,0.0,0.0,0.003,1.091,1.012,0.001,1.303,0.0,0.001,0.000,0.013,0.0,0.0
نسخته,0.0,0.0,0.0,0.002,0.000,0.000,0.005,0.000,0.0,0.001,1.011,0.000,0.0,0.0
الـ,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.0
أصلية,0.0,0.0,0.0,0.004,0.002,0.004,0.002,0.001,0.0,1.059,0.010,0.005,0.0,0.0
فقط,0.0,0.0,0.0,0.006,0.007,0.007,1.009,0.004,0.0,0.003,0.001,1.173,0.0,0.0


['(', 'b', ')', 'note', 'by', 'the', 'secretariat', '(', 'a', '/', '47', '/', '1002', ')', '.']
[')', 'ب', '(', 'مذكرة', 'من', 'الـ', 'أمانة', 'الـ', 'عامة', '(', 'a', '/', '47', '/', '1002', ')', '.']
[ 0  0  2  0  4  5  0  7  0  0  0 13  0 15  0  0 18]
[ 0  0  2  0  4  5  0  7  0  5  0 11  0 11  0 13  0  0 16]
-----
iteration # 0
iteration # 1
iteration # 2
iteration # 3
iteration # 4
iteration # 5
iteration # 6
iteration # 7
iteration # 8
iteration # 9
['<s>', '(', 'b', ')', 'note', 'by', 'the', 'secretariat', '(', 'a', '/', '47', '/', '1002', ')', '.', '</s>']
['<s>', ')', 'ب', '(', 'مذكرة', 'من', 'الـ', 'أمانة', 'الـ', 'عامة', '(', 'a', '/', '47', '/', '1002', ')', '.', '</s>']
[(0, 0), (2, 2), (4, 4), (5, 5), (7, 7), (13, 11), (15, 13), (18, 16)]
------


,<s>,(,b,),note,by,the,secretariat,(,a,/,47,/,1002,),.,</s>
<s>,2.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0
),0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0
ب,0.0,0.0,1.378,0.0,0.009,0.041,0.0,0.011,0.0,0.0,0.0,0.021,0.0,0.000,0.0,0.0,0.0
(,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0
مذكرة,0.0,0.0,0.006,0.0,1.252,0.014,0.0,0.016,0.0,0.0,0.0,0.013,0.0,0.002,0.0,0.0,0.0
من,0.0,0.0,0.014,0.0,0.010,1.125,0.0,0.011,0.0,0.0,0.0,0.016,0.0,0.000,0.0,0.0,0.0
الـ,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0
أمانة,0.0,0.0,0.009,0.0,0.021,0.020,0.0,1.812,0.0,0.0,0.0,0.014,0.0,0.001,0.0,0.0,0.0
الـ,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0
عامة,0.0,0.0,0.030,0.0,0.021,1.085,0.0,0.079,0.0,0.0,0.0,0.082,0.0,0.000,0.0,0.0,0.0


['the', 'candidate', "'s", 'curriculum', 'vitae', 'appears', 'in', 'the', 'annex', 'to', 'the', 'present', 'note', '.']
['وـ', 'ترد', 'في', 'مرفق', 'هذه', 'الـ', 'مذكرة', 'سيرة', 'الـ', 'مرشح', 'الـ', 'شخصية', '.']
[ 0  0 10  0  8  8  2  3  0  4  3  0  4  7  0 14]
[ 0  0  9  7  9  7  0 13  5  0  2  0  5  0 15]
-----
iteration # 0
iteration # 1
iteration # 2
iteration # 3
iteration # 4
iteration # 5
iteration # 6
iteration # 7
iteration # 8
iteration # 9
['<s>', 'the', 'candidate', "'s", 'curriculum', 'vitae', 'appears', 'in', 'the', 'annex', 'to', 'the', 'present', 'note', '.', '</s>']
['<s>', 'وـ', 'ترد', 'في', 'مرفق', 'هذه', 'الـ', 'مذكرة', 'سيرة', 'الـ', 'مرشح', 'الـ', 'شخصية', '.', '</s>']
[(0, 0), (2, 12), (3, 7), (4, 9), (5, 10), (7, 13), (8, 5), (10, 2), (12, 4), (14, 15)]
------


...

,<s>,the,council,concluded,its,consideration,on,21,may,1992,.,</s>
<s>,2.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.0
اختتم,0.0,0.0,0.000,1.060,0.0,0.007,0.000,0.001,0.001,0.001,0.0,0.0
الـ,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.0
مجلس,0.0,0.0,1.623,0.003,0.0,0.017,0.056,0.015,0.028,0.038,0.0,0.0
نظر,0.0,0.0,0.032,0.009,0.0,1.279,0.024,0.009,0.023,0.012,0.0,0.0
ـه,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.0
في,0.0,0.0,0.034,0.002,0.0,0.012,1.138,0.009,0.017,0.026,0.0,0.0
الـ,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.0
بند,0.0,0.0,0.016,0.007,0.0,1.059,0.024,0.011,0.014,0.008,0.0,0.0
في,0.0,0.0,0.034,0.002,0.0,0.012,1.138,0.009,0.017,0.026,0.0,0.0


['*', 'reissued', 'for', 'technical', 'reasons', '.']
['*', 'أعيد', 'إصدار', 'ـها', 'لـ', 'أسباب', 'فنية', '.']
[0 0 2 7 7 6 0 9]
[0 0 2 2 0 0 5 4 0 7]
-----
iteration # 0
iteration # 1
iteration # 2
iteration # 3
iteration # 4
iteration # 5
iteration # 6
iteration # 7
iteration # 8
iteration # 9
['<s>', '*', 'reissued', 'for', 'technical', 'reasons', '.', '</s>']
['<s>', '*', 'أعيد', 'إصدار', 'ـها', 'لـ', 'أسباب', 'فنية', '.', '</s>']
[(0, 0), (2, 2), (3, 3), (6, 5), (7, 4), (9, 7)]
------


,<s>,*,reissued,for,technical,reasons,.,</s>
<s>,2.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
*,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
أعيد,0.0,0.0,1.074,0.001,0.004,0.020,0.0,0.0
إصدار,0.0,0.0,1.028,1.003,0.005,0.019,0.0,0.0
ـها,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
لـ,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
أسباب,0.0,0.0,0.014,0.008,0.006,1.398,0.0,0.0
فنية,0.0,0.0,0.010,1.008,1.046,0.009,0.0,0.0
.,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
</s>,0.0,0.0,0.000,0.000,0.000,0.000,0.0,2.0


['a', 'total', 'of', '20', 'helicopters', 'would', 'also', 'be', 'needed', '.']
['وـ', 'سـ', 'تكون', 'هناك', 'حاجة', 'أيضا', 'إلى', 'ما', 'مجموع', 'ـه', '20', 'طائرة', 'هليكوبتر', '.']
[ 0  0  9  0 11 13  7  6  7  5  0 15]
[ 0  0  0  8  8  9  7  8  8  2  0  4  5  5  0 11]
-----
iteration # 0
iteration # 1
iteration # 2
iteration # 3
iteration # 4
iteration # 5
iteration # 6
iteration # 7


KeyboardInterrupt: ignored

In [ ]:
def get_opt_paths(cur_span_dict,composite_span_dict={}, se=True):
  prev_span_dict={} #prev_span_se_dict - initialize dictionaries
  prev_wt_dict={} #prev_wt_se_dict
   #the spans that each span consists of
  #list_spans  list_se_spans prev_wt_se_dict
  if se: list_spans=sorted(list(cur_span_dict.keys()),key=lambda x:x[0][0]) #sort the keys of the span dict towards SE
  else: list_spans=sorted(list(cur_span_dict.keys()),key=lambda x:(x[0][0],-x[0][1]) ) #sort the keys towards NE

  for li,cur_span in enumerate(list_spans): #going through each of the default span (towards SE)
    cur_span_wt=cur_span_dict.get(cur_span,0) #actual weight of the current span
    cur_span_path_wt=prev_wt_dict.get(cur_span,cur_span_wt) #cumulative weight of the path up to current span
    prev_wt_dict[cur_span]=cur_span_path_wt #we update the weight in the path/prev dict 
    span_pt1,span_pt2=cur_span
    pt2_x,pt2_y=span_pt2
    if se: next_spans=[(v_p1,v_p2) for v_p1,v_p2 in list_spans[li+1:] if v_p1[1]>=pt2_y] #identify all points towards SE from the current point
    else: next_spans=[(v_p1,v_p2) for v_p1,v_p2 in list_spans[li+1:] if v_p1[1]<=pt2_y] #identify all points towards NE from the current point
    for next_span in next_spans:    #iterate over each of these SE points
      next_span_wt=cur_span_dict.get(next_span,0) #get the actua weight of the next SE span
      combined_span_wt=cur_span_path_wt+next_span_wt #combine weight so far (path/prev weight) of the current span with the weight of next span
      found_combined_span_path_wt=prev_wt_dict.get(next_span,0) #check if there is a larger weight for the next span in the path/prev weight dictionary
      #print(cur_span,next_span,cur_span_path_wt,next_span_wt, "combined:",combined_span_wt, "found:", found_combined_span_path_wt )
      if combined_span_wt>found_combined_span_path_wt: #if the combined weight of the next span and path so far for current is greater than the path so far for next span
        prev_wt_dict[next_span]=combined_span_wt #we update the path of the weight so far for the next span
        prev_span_dict[next_span]=cur_span #and also update the current span as the previous point in the optimum path

  #all_se_paths
  all_paths=[] #now we get all the paths from each span till the beginning
  for k,v in prev_span_dict.items(): 
    found=v
    path=[k]
    if found: path.append(found)
    while found:
      found=prev_span_dict.get(found)
      if found: path.append(found)
    all_paths.append(path)
  for pth in all_paths: #now we will rectangle each two points on each path into new spans (both SE and NE), and get the highest weight for each span
    pth.reverse()
    wts=[cur_span_dict[v] for v in pth]
    for pi0 in range(len(pth)):
      for pi1 in range(pi0+1,len(pth)):
        span1,span2=pth[pi0],pth[pi1]
        full_span=(span1[0],span2[1])
        mixed_spans=span1+span2
        mixed_spans_xs=[v[0] for v in mixed_spans]
        mixed_spans_ys=[v[1] for v in mixed_spans]
        min_x,max_x=min(mixed_spans_xs),max(mixed_spans_xs)
        min_y,max_y=min(mixed_spans_ys),max(mixed_spans_ys)

        full_se_span=((min_x,min_y),(max_x,max_y))
        full_ne_span=((min_x,max_y),(max_x,min_y))
        cur_wts=wts[pi0:pi1+1]
        sum_wt=sum(cur_wts)
        #print(pi0,pi1,span1,span2,full_span,cur_wts)
        print(span1,span2,cur_wts)
        print("full_ne_span",full_ne_span, sum_wt)
        print("full_se_span", full_se_span, sum_wt)
        found_span_wt=cur_span_dict.get(full_se_span,0)
        if sum_wt>found_span_wt: 
          cur_span_dict[full_se_span]=sum_wt
          composite_span_dict[full_se_span]=pth[pi0:pi1+1]
        found_ne_span_wt=ne_spans_dict.get(full_ne_span,0)
        if sum_wt>found_ne_span_wt: ne_spans_dict[full_ne_span]=sum_wt

  return composite_span_dict,cur_span_dict

def get_rect_spans_OLD(candidate_pts, wt_list, n_iterations=10,max_skip=8): #perform dynamic programming on the list, by doing expanding rectangles
  candidate_pts.sort()
  cur_wt_dict=dict(iter(wt_list))
  #a span is the rectangle between two points [(0,0),(5,4)]
  span_dict={} #it has the weight of each span, updates with each iteration
  pt_dict={} #it has the points within this span, also updated with each iteration
  for i1,j1 in candidate_pts: #we initilize the span and pt dictionaries with the points in the current candidate list
    if i1==0 and j1!=0: continue
    if i1!=0 and j1==0: continue
    cur_span=((i1,j1),(i1,j1))
    #span_dict[cur_span]=array1[i1,j1]
    span_dict[cur_span]=cur_wt_dict[(i1,j1)]
    pt_dict[cur_span]=[(i1,j1)]
    #print(i1,j1)
    #array1[i1,j1]+=1 #just to keep track of the candidate points when visualized in the matrix plot
  full_span=((0,0),candidate_pts[-1]) #this is the span of the full matrix
  pts_combinations=[]
  prev_len=0 # to check if the num of spans in the current iteration is the same as the previous iteration
  for _ in range(n_iterations): #we do a number of iterations till the alignment converges across the matrix
    
    keys=sorted(span_dict.keys()) #check what spans we have so far
    print("iteration #", _, "number of keys:",len(keys))
    grouped_spans=[(key,list(group)) for key,group in groupby(keys,lambda x:x[0])]
    grouped_spans_dict=dict(iter(grouped_spans))
    # for sp,gs in grouped_spans[:10]:
    #   print(sp,len(gs), gs)
    # print(len(keys))
    if len(keys)==prev_len and pt_dict.get(full_span)!=None: break

    prev_len=len(keys)
    for ki,cur_span in enumerate(keys): #start iterating over spans, merge them to create new spans
      cur_wt=span_dict[cur_span]
      cur_pts=pt_dict[cur_span]
      cur_pt0,cur_pt1=cur_span
      cur_x0,cur_y0=cur_pt0
      cur_x1,cur_y1=cur_pt1
      all_next_spans=[]

      for inc_x in range(1,max_skip+1):
        for inc_y in range(1,max_skip+1):
          test_pt=(cur_x1+inc_x, cur_y1+inc_y) #test the surrounding points
          all_next_spans.extend(grouped_spans_dict.get(test_pt,[]))
          #print(">>>", cur_pt1, test_span_start_pt)
      #print(cur_pt1, len(all_next_spans))

      for next_span in all_next_spans: #checking the surrounding spans
      #for next_span in keys[ki+1:]: #checking the surrounding spans
        if cur_span[0][0]==next_span[0][0]: continue
        next_pt0,next_pt1=next_span
        next_x0,next_y0=next_pt0
        next_x1,next_y1=next_pt1
        if next_x0<=cur_x1: continue #spans expand to the right downwards
        if next_y0>=cur_y0 and next_y0<=cur_y1: continue
        if next_y1>=cur_y0 and next_y1<=cur_y1: continue
        if next_y0<=cur_y0 and next_y1>=cur_y1: continue

        next_wt=span_dict[next_span] #we get the weight and point info of the next span to be merged
        next_pts=pt_dict[next_span]
        combined_pts=sorted(cur_pts+next_pts)
        if combined_pts in pts_combinations: continue #when combining pts of cur and next spans, we ignore them if this combination was used before

        pts_combinations.append(combined_pts)
        x0=cur_span[0][0] #now constructing the new span
        x1=next_span[-1][0]
        y0=min(cur_span[0][1],next_span[0][1])
        y1=max(cur_span[-1][1],next_span[-1][1])
        new_span=((x0,y0),(x1,y1))
        new_wt=cur_wt+next_wt #and also the new weight
        new_pts=cur_pts+next_pts #and also the points in this span
        found_wt=span_dict.get(new_span,0) #but we also check if there is a weight for this span from previous iterations

        if new_wt> found_wt: #if the new weight of the merged span is larger than the found weight, we start updating the span_dict and pt_dict
          span_dict[new_span]=new_wt
          pt_dict[new_span]=combined_pts
  #solution=pt_dict[full_span] 
  solution=pt_dict.get(full_span,[]) #and the solution is the points that are in the pt_dict for the full span

  return solution #return the optimum span, from start to end


# cur_wt_dict=dict(iter(cur_wt_list))
# se_spans_dict={} #normal spans going towards south east  - diagonal
# ne_spans_dict= {} #spans going from towards noth east - opposite diagonal
# for p in pts:
#   #print(p, cur_wt_dict.get(p,0))
#   cur_wt=cur_wt_dict.get(p,0)
#   se_spans_dict[(p,p)]=cur_wt 
#   ne_spans_dict[(p,p)]=cur_wt

# prev_span_se_dict={} #now we start an algorithm based on dijkstra for getting the path of largest weight
# prev_span_ne_dict={}

# prev_wt_se_dict={}
# prev_wt_ne_dict={}

# #we start iterating
# #input span_dictionary, direction
# #cur_span_dict=se_spans_dict #ne_spans_dict
# composite_dict={}
# se=True
# #se=False

# composite_dict,se_spans_dict=get_opt_paths(se_spans_dict,composite_dict,se=True)
# #composite_dict,ne_spans_dict=get_opt_paths(ne_spans_dict,composite_dict,se=False)

# for a in composite_dict:
#   print(a, composite_dict[a], se_spans_dict.get(a,0) )


# # prev_span_dict={} #prev_span_se_dict
# # prev_wt_dict={} #prev_wt_se_dict
# # composite_span_dict={} #the spans that each span consists of

# # #list_spans  list_se_spans prev_wt_se_dict
# # if se: list_spans=sorted(list(cur_span_dict.keys()),key=lambda x:x[0][0]) #sort the keys of the span dict towards SE
# # else: list_spans=sorted(list(cur_span_dict.keys()),key=lambda x:(x[0][0],-x[0][1]) ) #sort the keys towards NE

# # for li,cur_span in enumerate(list_spans): #going through each of the default span (towards SE)
# #   cur_span_wt=cur_span_dict.get(cur_span,0) #actual weight of the current span
# #   cur_span_path_wt=prev_wt_dict.get(cur_span,cur_span_wt) #cumulative weight of the path up to current span
# #   prev_wt_dict[cur_span]=cur_span_path_wt #we update the weight in the path/prev dict 
# #   span_pt1,span_pt2=cur_span
# #   pt2_x,pt2_y=span_pt2
# #   if se: next_spans=[(v_p1,v_p2) for v_p1,v_p2 in list_spans[li+1:] if v_p1[1]>=pt2_y] #identify all points towards SE from the current point
# #   else: next_spans=[(v_p1,v_p2) for v_p1,v_p2 in list_spans[li+1:] if v_p1[1]<=pt2_y] #identify all points towards NE from the current point
# #   for next_span in next_spans:    #iterate over each of these SE points
# #     next_span_wt=cur_span_dict.get(next_span,0) #get the actua weight of the next SE span
# #     combined_span_wt=cur_span_path_wt+next_span_wt #combine weight so far (path/prev weight) of the current span with the weight of next span
# #     found_combined_span_path_wt=prev_wt_dict.get(next_span,0) #check if there is a larger weight for the next span in the path/prev weight dictionary
# #     #print(cur_span,next_span,cur_span_path_wt,next_span_wt, "combined:",combined_span_wt, "found:", found_combined_span_path_wt )
# #     if combined_span_wt>found_combined_span_path_wt: #if the combined weight of the next span and path so far for current is greater than the path so far for next span
# #       prev_wt_dict[next_span]=combined_span_wt #we update the path of the weight so far for the next span
# #       prev_span_dict[next_span]=cur_span #and also update the current span as the previous point in the optimum path

# # #all_se_paths
# # all_paths=[] #now we get all the paths from each span till the beginning
# # for k,v in prev_span_dict.items(): 
# #   found=v
# #   path=[k]
# #   if found: path.append(found)
# #   while found:
# #     found=prev_span_dict.get(found)
# #     if found: path.append(found)
# #   all_paths.append(path)
# # for pth in all_paths: #now we will rectangle each two points on each path into new spans (both SE and NE), and get the highest weight for each span
# #   pth.reverse()
# #   wts=[cur_span_dict[v] for v in pth]
# #   for pi0 in range(len(pth)):
# #     for pi1 in range(pi0+1,len(pth)):
# #       span1,span2=pth[pi0],pth[pi1]
# #       full_span=(span1[0],span2[1])
# #       mixed_spans=span1+span2
# #       mixed_spans_xs=[v[0] for v in mixed_spans]
# #       mixed_spans_ys=[v[1] for v in mixed_spans]
# #       min_x,max_x=min(mixed_spans_xs),max(mixed_spans_xs)
# #       min_y,max_y=min(mixed_spans_ys),max(mixed_spans_ys)

# #       full_se_span=((min_x,min_y),(max_x,max_y))
# #       full_ne_span=((min_x,max_y),(max_x,min_y))
# #       cur_wts=wts[pi0:pi1+1]
# #       sum_wt=sum(cur_wts)
# #       #print(pi0,pi1,span1,span2,full_span,cur_wts)
# #       print(span1,span2,cur_wts)
# #       print("full_ne_span",full_ne_span, sum_wt)
# #       print("full_se_span", full_se_span, sum_wt)
# #       found_span_wt=cur_span_dict.get(full_se_span,0)
# #       if sum_wt>found_span_wt: 
# #         cur_span_dict[full_se_span]=sum_wt
# #         composite_span_dict[full_se_span]=pth[pi0:pi1+1]
# #       found_ne_span_wt=ne_spans_dict.get(full_ne_span,0)
# #       if sum_wt>found_ne_span_wt: ne_spans_dict[full_ne_span]=sum_wt
      
      

# #       print("------")

# #   print(pth)
# #   print(wts)
# #   print("======")


#   #se_spans2=[(v_p1,v_p2) for v_p1,v_p2 in list_spans if v_p1[1]>=pt2_y and v_p1[0]>=pt2_x and (v_p1,v_p2)!=ls]
#   #print(len(se_spans), len(se_spans2), se_spans)
#   #print("-----")

# # list_up_spans=sorted(list(up_spans_dict.keys()),key=lambda x:(x[0][0],-x[0][1]) )
# # print(list_up_spans)
# # for li,ls in enumerate(list_up_spans):
# #   print(ls)
# #   span_pt1,span_pt2=ls
# #   pt2_x,pt2_y=span_pt2
# #   ne_spans=[(v_p1,v_p2) for v_p1,v_p2 in list_up_spans[li+1:] if v_p1[1]<=pt2_y]
# #   #ne_spans2=[(v_p1,v_p2) for v_p1,v_p2 in list_up_spans if v_p1[1]<=pt2_y and v_p1[0]>=pt2_x and (v_p1,v_p2)!=ls]
# #   #print(len(ne_spans), len(ne_spans2), ne_spans)
# #   print("-----")


# # cur_phrase_pairs=align_extract(src,trg,src_inverted,trg_inverted)

# # for cp in cur_phrase_pairs:
# #   print(cp) 



In [ ]:
import sys
sys.version

'3.6.9 (default, Apr 18 2020, 01:56:04) \n[GCC 8.4.0]'

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://siq1talaiy-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [ ]:
!python -m http.server 8000

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [23/Jul/2020 13:53:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 13:53:25] code 404, message File not found
127.0.0.1 - - [23/Jul/2020 13:53:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2020 14:01:47] "GET /web HTTP/1.1" 301 -
127.0.0.1 - - [23/Jul/2020 14:01:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 14:01:57] code 404, message File not found
127.0.0.1 - - [23/Jul/2020 14:01:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2020 14:02:02] "GET /web/ HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 14:02:03] code 404, message File not found
127.0.0.1 - - [23/Jul/2020 14:02:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2020 14:04:14] "GET /web/ HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 14:04:15] code 404, message File not found
127.0.0.1 - - [23/Jul/2020 14:04:15] "GET /favicon.ico HTTP/1.1" 404 -


In [55]:
#https://github.com/hmghaly/word_align.git
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/code")
!git init
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
!git pull --rebase git@github.com:hmghaly/word_align.git master
!git add .
!git commit -m "Update existing file"
!git push --mirror git@github.com:hmghaly/word_align.git


Reinitialized existing Git repository in /content/drive/My Drive/Colab Notebooks/code/.git/
# github.com:22 SSH-2.0-babeld-5a455904
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From github.com:hmghaly/word_align
 * branch            master     -> FETCH_HEAD
Updating 17ecb74..4182329
Fast-forward
 README.md | 2 ++
 1 file changed, 2 insertions(+)
 create mode 100644 README.md
Current branch master is up to date.
On branch master
nothing to commit, working tree clean
Everything up-to-date


In [17]:
!ssh-keygen -t rsa -b 4096 -C "hmghaly@gmail.com"
!cat /root/.ssh/id_rsa.pub

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): /root/.ssh/id_rsa
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:sgifENXbxaRsY3HUbbMiqERUi5D4t5O5x8ehqYsIdWI hmghaly@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|   .o+..o++. .   |
|  ....oo =o . +  |
|  .. ..oBo   . o |
|   .. +oo.. . .  |
|  E .o.=S  . .   |
| o * o*o  .      |
|.   + .+ + .     |
| . . .. = o      |
|  . . o+ .       |
+----[SHA256]-----+


In [19]:

!cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDmvIsBhP0BBkD/lCLcMIRigcI/ytomuDsRvxb9uvcO+pRXBB3F+R1OoJu9NmYOsCn+gpjLdIURMNzkybSbcd1pXHed4GV8q1OTnVXM9QjzD/8HzvXPGPjY2M0OiZw2QpU4ZGh+U6wuj+oeT3URdxutUNiOaVvgVMV1/WH4IuMZDArD548l6r+yPl2PXuQfL8T5hZyWS9kKnJ/gogYJjPAIgZBtg0a7Dy1+tVr6LZJQPLsvdIC1brBQL+bnt+5vyLWunUzcr878DXAKS8UFoZfTlZ8o5cdvODSP7cneWCOmUOad9KnvGTu93ncXCRlByA1LHxtm19OK7RvJfnJD05rAgXS/kR2rqlGrLtX2i4+zDMhmdRBWcSyO1cirA/F6QzFW3JUyLB9pZITdSotVVquu2q5eEGZm2kWUc0eyj1rXmCABdIJomCZWs18FlaEgTbFoUXWznIMriHzVMY74FZCAOUr4rvQsUA3XRMCdtYqkieGe8ZSNInknGUrMwWjeZzeH+feZ4ipRf2vQWv15/eUXYy5jz7T+UNRaHRtnEvx0TzXaHmSLpi6ZpHQwoaRKSYSrid8/EVp5m5Vul4xqeJWv+AF1IA4eJ5eOAxoA6t48HDPdNmjnmCMDpIlV+VhUaRXFLAMzzvvyuRQMP58VHLFT7ayBv5QwvpJGkVa3NRAG8Q== hmghaly@gmail.com


In [35]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/code")
!git init


Reinitialized existing Git repository in /content/drive/My Drive/Colab Notebooks/code/.git/
